<a href="https://colab.research.google.com/github/tejomaygadgil/cgpos/blob/nn/notebooks/colab/1_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b nn https://github.com/tejomaygadgil/cgpos.git
!pip install git+https://github.com/jtauber/greek-accentuation.git beta-code
%cd cgpos/
!make process_ft_data

Cloning into 'cgpos'...
remote: Enumerating objects: 2199, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 2199 (delta 117), reused 127 (delta 71), pack-reused 2022
Receiving objects: 100% (2199/2199), 101.85 MiB | 35.89 MiB/s, done.
Resolving deltas: 100% (1363/1363), done.
  Cloning https://github.com/jtauber/greek-accentuation.git to /tmp/pip-req-build-p2uil6ms
  Running command git clone --filter=blob:none --quiet https://github.com/jtauber/greek-accentuation.git /tmp/pip-req-build-p2uil6ms
  Resolved https://github.com/jtauber/greek-accentuation.git to commit 15ac5fd1cc82c8f9b91a4041f9b64399c9552097
  Preparing metadata (setup.py) ... done
  Created wheel for greek-accentuation: filename=greek_accentuation-1.2.0-py2.py3-none-any.whl size=7639 sha256=3410f14c88d242b2cabc2cb7bda2562c38940bedf852caa806018f9a11fbb605
  Stored in directory: /tmp/pip-ephem-wheel-cache-84nt0gf8/wheels/31/68/a9/73949cf9e13b5f173b691a48

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%cd ../../

/home


In [14]:
import random 

import torch
import torch.nn as nn

from src.model import Transformer
import src.config as cfg
from src.util import read_pkl, encode, decode, get_batch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
n_head = 8
max_iters = 1000
n_eval = 20

params = {
    "train_size": 0.98,  # Train params
    "n_chunks": 500,
    "unc_rate": 0.005,
    "device": "cuda" if torch.cuda.is_available() else "cpu",  # Device params
    "batch_size": 64,  # Model hyperparameters
    "block_size": 256,
    "n_head": n_head,
    "emb_size": 64 * n_head,
    "n_layer": 6,
    "dropout": 0.6,  # Training hyperparameters
    "max_iters": max_iters,
    "eval_interval": max_iters // n_eval,
    "learning_rate": 3e-4,
    "eval_iters": 200,  # Monitor settings
    "generate_len": 32,
    "torch_seed": 20,  # Seeds
    "random_seed": 40,
    }

for key, value in params.items():
    globals()[key] = value

wandb.init(
    project="ncgpos_ft",
    config=params,
)

In [5]:
ft_syl = read_pkl(cfg.ft_syl)
ft_targets = read_pkl(cfg.ft_targets)
ft_targets_map = read_pkl(cfg.ft_targets_map)
assert len(ft_syl) == len(ft_targets)

In [9]:
tokens = []
labels = []
for i, word in enumerate(ft_syl):
    for token in word:
        tokens.append(token)
        labels.append(ft_targets[i][0])

In [44]:
model = Transformer(
    vocab_size=17016, 
    block_size=256, 
    emb_size=512, 
    n_layer=6, 
    n_head=8, 
    dropout=0.6, 
    device=device
)
model.load_state_dict(torch.load(cfg.wts, map_location=torch.device(device)))
# Omg!!
model.lm_head = nn.Linear(512, len(ft_targets_map[1][0]))

m = model.to(device)

In [ ]:
@torch.no_grad()
def estimate_loss(eval_iters, device, *batch_args):
    out = []
    model.eval()
    for data in [train_data, val_data]:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(data, *batch_args)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out.append(losses.mean())
    model.train()
    return out


In [28]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for step in tqdm(range(max_iters)):
    # Evaluate training and val loss every eval_interval
    if (step % eval_interval == 0) or (iter == max_iters - 1):
        train_loss, val_loss = estimate_loss(
            eval_iters, device, block_size, batch_size, device
        )
        wandb.log({"train_loss": train_loss, "val_loss": val_loss})
        with logging_redirect_tqdm():
            logging.info(
                f"step {step}: train loss {train_loss:.4f}, val loss {val_loss:.4f}"
            )
            logging.info(generate(generate_len, block_size, model, device))

    # Sample batch
    xb, yb = get_batch(train_data, block_size, batch_size, device)
    _, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

wandb.finish()

Linear(in_features=512, out_features=16970, bias=True)